In [1]:
from langchain_groq import ChatGroq

# TODO: improve all prompts
# TODO: the model is going to get deprecated soon so look for alternates that perform well. 
llm = ChatGroq(model="mixtral-8x7b-32768")

In [2]:
from typing import List

from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field


def parse_pdf(path: str) -> str:
    docs = PyPDFLoader(path).load()
    text = " ".join(doc.page_content for doc in docs)
    return text


class Experience(BaseModel):
    """Schema for an experience entry on resume."""

    title: str = Field(description="The title of the job.")
    description: str = Field(description="A description of the job.")


class Project(BaseModel):
    """Schema for a project entry on resume."""

    title: str = Field(description="The title of the project.")
    description: str = Field(description="A description of the project.")


class Resume(BaseModel):
    """Breakdown of a resume into experiences, projects, and skills."""

    experiences: List[Experience] = Field(
        description="The experiences listed on the resume."
    )
    projects: List[Project] = Field(description="The projects listed on the resume.")
    skills: List[str] = Field(description="The skills listed on the resume.")


resume_breaker_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a resume parser. Your task is to breakdown the resume provided "
            "below into three sections: experiences, projects, and skills.",
        ),
        ("system", "<resume>\n{resume}\n</resume>"),
    ]
)
resume_breaker = resume_breaker_prompt | llm.with_structured_output(Resume)

In [3]:
# resume = resume_breaker.invoke({"resume": parse_pdf("./media/resume.pdf")})
# resume

In [4]:
class CodingQuestions(BaseModel):
    """Schema for a coding question."""

    questions: List[str] = Field(description="The list of technical coding questions.")


# TODO: use few shot prompts with this system prompt and differnt techstack and questions related to it.
# You can use job description as well if given in the state.
generate_coding_question_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a coding interviewer. Your task is to generate exactly 5 technical "
            "programming/coding questions based on the skills mentioned below."
        ),
        ("system", "<skills>\n{skills}\n</skills>"),
    ]
)
generate_coding_question = generate_coding_question_prompt | llm.with_structured_output(
    CodingQuestions
)

In [5]:
# skills = ", ".join(skill for skill in resume.skills)
# coding_questions = generate_coding_question.invoke({"skills": skills})
# coding_questions

In [6]:
class CodingInterviewScore(BaseModel):
    """Schema for a coding interview score."""

    score: int = Field(description="The score of the candidate's response.")


coding_question_assessment_promt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a coding interviewer. Your task is to assess the candidate's "
            "response by a score between 1 to 10 where 1 is very bad and 10 is perfect "
            "answer. Here is the question: \n{question}",
        ),
        ("user", "{response}"),
    ]
)
coding_question_assessment = (
    coding_question_assessment_promt | llm.with_structured_output(CodingInterviewScore)
)

In [7]:
from typing import Union

class ExperienceInterviewScore(BaseModel):
    """Schema for an experience interview score."""

    score: int = Field(
        description=(
            "The score of the candidate's responses between 1 to 10 where 1 is "
            "extremely bad and 10 is perfect."
        )
    )


class ExperienceInterviewQuestion(BaseModel):
    """Schema for an experience interview."""

    response: Union[str, ExperienceInterviewScore] = Field(
    # response: str = Field(
        description=(
            "The question you want to ask. If you want to stop the conversation, "
            "provide the score to candidate's responses using the Experience "
            "Interview score schema."
        )
    )


expereince_interviewer_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an highly experienced interviewer at a tech company and your task is to take "
            "interviews of new candidatew for the provided job description (if any is provided). You "
            "can ask questions about the candidate's previous work experiences. If you think that you "
            "need to ask a follow up question, you can ask that as well or else you can move on to the "
            "next question. No need to extend you conversation. Ask only relevant questions. Do not "
            "call any tools",
        ),
        ("system", "<experience>\n{experience}\n</experience>"),
        ("system", "<projects>\n{projects}\n</projects>"),
    ]
)

experience_interviewer = expereince_interviewer_prompt | llm.with_structured_output(
    ExperienceInterviewQuestion
)

In [8]:
# experience_interviewer.invoke(
#     {
#         "experience": resume.experiences,
#         "projects": resume.projects,
#     }
# )

In [9]:
from langgraph.graph import MessagesState


class Interview(MessagesState):
    job_description: str
    experiences: List[Experience]
    projects: List[Project]
    skills: List[str]
    coding_interview_score: int
    experience_interview_score: int

In [10]:
from langgraph.types import Command, interrupt
from langgraph.graph import END


async def resume_parser_node(state: Interview):
    resume = parse_pdf(
        "../media/resume.pdf"
    )  # TODO: we can make it a task using @task in the node ig.
    parsed_resume = await resume_breaker.ainvoke({"resume": resume})
    return {
        "experiences": parsed_resume.experiences,
        "projects": parsed_resume.projects,
        "skills": parsed_resume.skills,
    }


# TODO: implement it like the experience_interviewer_node if that works
async def coding_interviewer_node(state: Interview):
    skills = ", ".join(skill for skill in state["skills"])
    coding_questions = await generate_coding_question.ainvoke({"skills": skills})
    questions = coding_questions.questions
    total_score = 10
    # TODO: uncomment it
    for question in questions:
        answer = interrupt(value=f"Answer the following: {question}\n")
        score = await coding_question_assessment.ainvoke(
            {"question": question, "response": answer}
        )
        total_score += score.score
    return {
        "coding_interview_score": total_score // len(questions),
    }


async def experience_interviewer_node(state: Interview):
    response = await experience_interviewer.ainvoke(
        {"experience": state["experiences"], "projects": state["projects"]}
    )
    if isinstance(response, ExperienceInterviewScore):
        return {"experience_interview_score": response.score}
    return Command(
        update={"messages": [{"role": "ai", "content": response.response}]},
        goto="candidate_node",
    )


def candidate_node(state: Interview):
    answer = interrupt("Answer the question. \n")
    return Command(
        update={"messages": [{"role": "human", "content": answer}]},
        goto="experience_interviewer_node",
    )


def should_end(state: Interview):
    if "experience_interview_score" in state and state["experience_interview_score"]:
        return END
    else:
        return "candidate_node"

In [11]:
from langgraph.graph import StateGraph, START
from langgraph.checkpoint.memory import MemorySaver

workflow = StateGraph(Interview)

workflow.add_node("resume_parser_node", resume_parser_node)
workflow.add_node("coding_interviewer_node", coding_interviewer_node)
workflow.add_node("experience_interviewer_node", experience_interviewer_node)
workflow.add_node("candidate_node", candidate_node)


workflow.add_edge(START, "resume_parser_node")
workflow.add_edge("resume_parser_node", "coding_interviewer_node")
workflow.add_edge("coding_interviewer_node", "experience_interviewer_node")
workflow.add_edge("candidate_node", "experience_interviewer_node")
workflow.add_conditional_edges(
    "experience_interviewer_node", should_end, ["candidate_node", END]
)

graph = workflow.compile(checkpointer=MemorySaver())

In [12]:
# from IPython.display import Image, display

# display(Image(graph.get_graph(xray=True).draw_mermaid_png()))

In [13]:
config = {"configurable": {"thread_id": "aayush1"}}

async for update in graph.astream(
    {
        "job_description": """
We are seeking a highly skilled AI Developer with expertise in Llama, OpenAI, and Falcon models to join our team. The ideal candidate will have strong experience in developing, fine-tuning, and deploying AI models, working with large language models (LLMs), and integrating AI solutions into real-world applications.

Key Responsibilities:
• Develop, fine-tune, and deploy AI/ML models using Llama, OpenAI, and Falcon.
• Optimize and customize pre-trained language models for specific use cases.
• Implement AI-powered solutions for chatbots, automation, and text generation.
• Integrate AI models into web and cloud-based applications.
• Work with APIs and SDKs from OpenAI, Meta (Llama), and Falcon to build scalable AI solutions.
• Perform data preprocessing, model training, and performance evaluation.
• Stay up to date with the latest advancements in natural language processing (NLP) and LLMs.
• Collaborate with cross-functional teams to deliver AI-driven projects.
• Troubleshoot and enhance model performance using hyperparameter tuning and optimization techniques.
• Ensure compliance with AI ethics, security, and data privacy standards.

Required Skills & Qualifications:
• Experience between 3-6 yrs in AI/ML development, with hands-on experience in Llama, OpenAI (GPT models), and Falcon.
• Strong programming skills in Python, with experience in TensorFlow, PyTorch, or Hugging Face Transformers.
• Proficiency in API development and integration with AI models.
• Experience in fine-tuning large language models (LLMs) for various applications.
• Knowledge of cloud platforms (AWS, GCP, or Azure) for AI model deployment.
• Hands-on experience with NLP techniques, embeddings, and tokenization.
• Strong understanding of vector databases (e.g., Pinecone, FAISS) for efficient AI retrieval tasks.
• Experience in optimizing AI models for speed, accuracy, and scalability.
• Familiarity with Docker and Kubernetes for deploying AI applications.
• Strong problem-solving skills and ability to work in an agile development environment.

Preferred Qualifications:
• Experience working with LangChain and RAG-based (Retrieval-Augmented Generation) AI models.
• Knowledge of MLOps practices for model lifecycle management.
• Experience with Graph Neural Networks (GNNs) or multimodal AI models.
• Background in AI security and bias mitigation."""
    },
    config=config,
    # stream_mode="debug",
    debug = True
):
    print(update)
    print(
        "----------------------------------------------------------------------------------------------------------------"
    )

[-1:checkpoint] State at the end of step -1:
{'messages': []}
[0:tasks] Starting 1 task for step 0:
- __start__ -> {'job_description': '\n'
                    'We are seeking a highly skilled AI Developer with '
                    'expertise in Llama, OpenAI, and Falcon models to join our '
                    'team. The ideal candidate will have strong experience in '
                    'developing, fine-tuning, and deploying AI models, working '
                    'with large language models (LLMs), and integrating AI '
                    'solutions into real-world applications.\n'
                    '\n'
                    'Key Responsibilities:\n'
                    '• Develop, fine-tune, and deploy AI/ML models using '
                    'Llama, OpenAI, and Falcon.\n'
                    '• Optimize and customize pre-trained language models for '
                    'specific use cases.\n'
                    '• Implement AI-powered solutions for chatbots, '
             

BadRequestError: Error code: 400 - {'error': {'message': 'The model `mixtral-8x7b-32768` has been decommissioned and is no longer supported. Please refer to https://console.groq.com/docs/deprecations for a recommendation on which model to use instead.', 'type': 'invalid_request_error', 'code': 'model_decommissioned'}}

In [ ]:
async for event in graph.astream(
    Command(resume="I built a web scraper using selenium that scraped data from jobscan and i used the generated data to to tailor the resume for the job i was applying for."),
    # "I built a web scraper using selenium that scraped data from jobscan and i used the generated data to to tailor the resume for the job i was applying for.",
    config=config,
    # stream_mode="debug",
):
    print(event)
    print(
        "----------------------------------------------------------------------------------------------------------------"
    )


In [ ]:
graph.get_state(config=config)